In [ ]:
#!pip install neurokit2
#!pip install xmltodict

In [1]:
import os
import torch
import torch.nn as nn
import pickle
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import base64
import xmltodict
import array
import json

In [2]:
def ecgxmlparser(direc): #only rhythm type of wave
    with open(direc, 'r', encoding='ISO-8859-1') as xml:
        data = xmltodict.parse(xml.read())
    waveformtype = type(data['RestingECG']['Waveform'])
    wavevals = []
    ids = []
    if waveformtype != type([]):
        for i in range(len(data['RestingECG']['Waveform']['LeadData'])):
            if data['RestingECG']['Waveform']['LeadData'][i]['WaveFormData']!=None:
                lead_b64 = base64.b64decode(data['RestingECG']['Waveform']['LeadData'][i]['WaveFormData'])
                leadid = data['RestingECG']['Waveform']['LeadData'][i]['LeadID']
                lead_vals = np.array([])
                length = len(lead_b64)
                for n in range(0, length):
                    try:
                        lead_vals = np.array(array.array('h', lead_b64[:length-n]))
                        break
                    except :
                        continue
                ids.append(leadid)
                wavevals.append(lead_vals)
            else:
                leadid = data['RestingECG']['Waveform']['LeadData'][i]['LeadID']
                ids.append(leadid)
                wavevals.append(np.array([]))
    elif waveformtype == type([]):
        wr = np.where(np.array([i['WaveformType'] for i in data['RestingECG']['Waveform']])=='Rhythm')[0][0]
        for i in range(len(data['RestingECG']['Waveform'][wr]['LeadData'])):
            if data['RestingECG']['Waveform'][wr]['LeadData'][i]['WaveFormData']!=None:
                lead_b64 = base64.b64decode(data['RestingECG']['Waveform'][wr]['LeadData'][i]['WaveFormData'])
                leadid = data['RestingECG']['Waveform'][wr]['LeadData'][i]['LeadID']
                lead_vals = np.array([])
                length = len(lead_b64)
                for n in range(0, length):
                    try:
                        lead_vals = np.array(array.array('h', lead_b64[:length-n]))
                        break
                    except :
                        continue
                ids.append(leadid)
                wavevals.append(lead_vals)
            else :
                leadid = data['RestingECG']['Waveform'][wr]['LeadData'][i]['LeadID']
                ids.append(leadid)
                wavevals.append(np.array([]))
            
    leads = dict()
    n=0
    for j in ids:
        leads[j]=wavevals[n]
        n+=1
    if ('I' in ids) & ('II' in ids) & ('III' not in ids):
        if (len(leads['I'])!=0) & (len(leads['II'])!=0) & (len(leads['I'])==len(leads['II'])):
            leads['III'] = np.subtract(leads['II'], leads['I'])
        else : leads['III'] = np.array([])
    elif ('III' in ids): 
        leads['III'] = leads['III']
    else : leads['III'] = np.array([])
        
    if ('I' in ids) & ('II' in ids) & ('aVR' not in ids):
        if (len(leads['I'])!=0) & (len(leads['II'])!=0) & (len(leads['I'])==len(leads['II'])):
            leads['aVR'] = np.add(leads['I'], leads['II'])*(-0.5)
        else : leads['aVR'] = np.array([])
    elif ('aVR' in ids): 
        leads['aVR'] = leads['aVR']
    else : leads['aVR'] = np.array([])

    if ('I' in ids) & ('II' in ids) & ('aVL' not in ids):
        if (len(leads['I'])!=0) & (len(leads['II'])!=0) & (len(leads['I'])==len(leads['II'])):
            leads['aVL'] = np.subtract(leads['I'], 0.5*leads['II'])
        else : leads['aVL'] = np.array([])
    elif ('aVL' in ids): 
        leads['aVL'] = leads['aVL']
    else : leads['aVL'] = np.array([])
   
    if ('I' in ids) & ('II' in ids) & ('aVF' not in ids):
        if (len(leads['I'])!=0) & (len(leads['II'])!=0) & (len(leads['I'])==len(leads['II'])):
            leads['aVF'] = np.subtract(leads['II'], 0.5*leads['I'])
        else : leads['aVF'] = np.array([])
    elif ('aVF' in ids): 
        leads['aVF'] = leads['aVF']
    else : leads['aVF'] = np.array([])
        
    if ('aVR' in ids) & ('aVL' in ids) & ('I' not in ids):
        if (len(leads['aVR'])!=0) & (len(leads['aVL'])!=0) & (len(leads['aVR'])==len(leads['aVL'])):
            leads['I'] = np.subtract(leads['aVR'], leads['aVL'])/(-1.5)
        else : leads['I'] = np.array([])
    elif ('I' in ids): 
        leads['I'] = leads['I']
    else : leads['I'] = np.array([])
        
    if ('aVR' in ids) & ('aVL' in ids) & ('II' not in ids):
        if (len(leads['aVR'])!=0) & (len(leads['aVL'])!=0) & (len(leads['aVR'])==len(leads['aVL'])):
            leads['II'] = np.subtract(-2*leads['aVR'], leads['aVL'])/(1.5)
        else : leads['II'] = np.array([])
    elif ('II' in ids): 
        leads['II'] = leads['II']
    else : leads['II'] = np.array([])
        
    if (len(leads['I'])!=0) & (len(leads['II'])!=0) & (len(leads['I'])==len(leads['II'])) & (len(leads['III'])==0):
        leads['III'] = np.subtract(leads['II'], leads['I'])
            
    return leads  

In [3]:
# Data가 있는 디렉토리 설정, train, validation 폴더가 있는 폴더
datadir = 'C:/Users/minso/Documents/electrocardiogram/data/'
#label 폴더 디텍토리 설정
labdir = 'C:/Users/minso/Documents/electrocardiogram/label/'

In [5]:
aVRs = []
aVFs = []
aVLs = []
Is = []
IIs = []
IIIs = []
labels = []
fn = []
tvind = []
for i in ['train', 'validation']:
    newdir = datadir + i + '/'
    for j in ['arrhythmia', 'normal']:
        newdir2 = newdir + j +'/'
        for o in os.listdir(newdir2):
            labelin = labdir + i + '/' + j + '/' + o[:-4] + '.json'
            parsed = ecgxmlparser(newdir2+o)
            parsedI = parsed['I']
            parsedII = parsed['II']
            parsedIII = parsed['III']
            parsedaVR = parsed['aVR']            
            parsedaVF = parsed['aVF']            
            parsedaVL = parsed['aVL']                     
            with open(labelin,'rb') as f:
                jfile = json.load(f)
            label = jfile['labels'][0]['label_id']
            aVRs.append(parsedaVR)
            aVFs.append(parsedaVF)
            aVLs.append(parsedaVL)
            Is.append(parsedI)
            IIs.append(parsedII)
            IIIs.append(parsedIII)
            labels.append(label)
            tvind.append(i)

#### For 6 leads

In [7]:
np.unique(np.array([len(i) for i in Is]))

array([   0, 1176, 1249, 4440, 4999, 5000])

In [8]:
np.unique(np.array([len(i) for i in IIs]))

array([   0, 1249, 4999, 5000])

In [9]:
np.unique(np.array([len(i) for i in IIIs]))

array([   0, 1249, 4999, 5000])

In [10]:
np.unique(np.array([len(i) for i in aVRs]))

array([   0, 1249, 4999, 5000])

In [11]:
np.unique(np.array([len(i) for i in aVLs]))

array([   0, 1249, 4999, 5000])

In [12]:
np.unique(np.array([len(i) for i in aVFs]))

array([   0, 1249, 4999, 5000])

In [13]:
#length가 상이한 example 제거
exceptionavrs = \
np.where(np.array([len(i) for i in Is])==0)[0].tolist()+\
np.where(np.array([len(i) for i in Is])==1176)[0].tolist()+\
np.where(np.array([len(i) for i in Is])==1249)[0].tolist()+\
np.where(np.array([len(i) for i in Is])==4440)[0].tolist()+\
np.where(np.array([len(i) for i in IIs])==0)[0].tolist()+\
np.where(np.array([len(i) for i in IIs])==1249)[0].tolist()+\
np.where(np.array([len(i) for i in IIIs])==0)[0].tolist()+\
np.where(np.array([len(i) for i in IIIs])==1249)[0].tolist()+\
np.where(np.array([len(i) for i in aVRs])==0)[0].tolist()+\
np.where(np.array([len(i) for i in aVRs])==1249)[0].tolist()+\
np.where(np.array([len(i) for i in aVLs])==0)[0].tolist()+\
np.where(np.array([len(i) for i in aVLs])==1249)[0].tolist()+\
np.where(np.array([len(i) for i in aVFs])==0)[0].tolist()+\
np.where(np.array([len(i) for i in aVFs])==1249)[0].tolist()
exceptionavrs = np.unique(exceptionavrs).tolist()

In [14]:
exceptionavrs = [i for i in range(len(aVRs)) if i not in exceptionavrs]

In [15]:
elabels = np.array(labels)[np.array(exceptionavrs)]
etvind = np.array(tvind)[np.array(exceptionavrs)]

In [16]:
#warning 무시
eIs = np.array(Is)[np.array(exceptionavrs)]
eIIs = np.array(IIs)[np.array(exceptionavrs)]
eIIIs = np.array(IIIs)[np.array(exceptionavrs)]
eaVRs = np.array(aVRs)[np.array(exceptionavrs)]
eaVLs = np.array(aVLs)[np.array(exceptionavrs)]
eaVFs = np.array(aVFs)[np.array(exceptionavrs)]

<ipython-input-16-15e9727fb5d5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  eIs = np.array(Is)[np.array(exceptionavrs)]
<ipython-input-16-15e9727fb5d5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  eIIs = np.array(IIs)[np.array(exceptionavrs)]
<ipython-input-16-15e9727fb5d5>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  e

In [17]:
#cleaning
cIs = [nk.ecg_clean(i, sampling_rate=500) for i in eIs]
cIIs = [nk.ecg_clean(i, sampling_rate=500) for i in eIIs]
cIIIs = [nk.ecg_clean(i, sampling_rate=500) for i in eIIIs]
caVRs = [nk.ecg_clean(i, sampling_rate=500) for i in eaVRs]
caVFs = [nk.ecg_clean(i, sampling_rate=500) for i in eaVFs]
caVLs = [nk.ecg_clean(i, sampling_rate=500) for i in eaVLs]

In [18]:
def padding(seq):
    if len(seq)==5000:
        return seq
    else :
        padlen = 5000-len(seq)
        v = np.mean(seq)
        seq = np.concatenate([seq,np.array([v]*padlen)])
        return seq

In [19]:
cIs = np.stack([padding(i) for i in cIs])
cIIs = np.stack([padding(i) for i in cIIs])
cIIIs = np.stack([padding(i) for i in cIIIs])
caVRs = np.stack([padding(i) for i in caVRs])
caVLs = np.stack([padding(i) for i in caVLs])
caVFs = np.stack([padding(i) for i in caVFs])

In [20]:
train_cIs = cIs[etvind=='train']
test_cIs = cIs[etvind=='validation']
train_cIIs = cIIs[etvind=='train']
test_cIIs = cIIs[etvind=='validation']
train_cIIIs = cIIIs[etvind=='train']
test_cIIIs = cIIIs[etvind=='validation']
train_caVRs = caVRs[etvind=='train']
test_caVRs = caVRs[etvind=='validation']
train_caVLs = caVLs[etvind=='train']
test_caVLs = caVLs[etvind=='validation']
train_caVFs = caVFs[etvind=='train']
test_caVFs = caVFs[etvind=='validation']

In [21]:
train_elabels = elabels[etvind=='train']
test_elabels = elabels[etvind=='validation']

In [22]:
np.save('train_moreleads_noseg_Is3.npy',train_cIs)
np.save('test_moreleads_noseg_Is3.npy',test_cIs)
np.save('train_moreleads_noseg_IIs3.npy',train_cIIs)
np.save('test_moreleads_noseg_IIs3.npy',test_cIIs)
np.save('train_moreleads_noseg_IIIs3.npy',train_cIIIs)
np.save('test_moreleads_noseg_IIIs3.npy',test_cIIIs)
np.save('train_moreleads_noseg_aVRs3.npy',train_caVRs)
np.save('test_moreleads_noseg_aVRs3.npy',test_caVRs)
np.save('train_moreleads_noseg_aVLs3.npy',train_caVLs)
np.save('test_moreleads_noseg_aVLs3.npy',test_caVLs)
np.save('train_moreleads_noseg_aVFs3.npy',train_caVFs)
np.save('test_moreleads_noseg_aVFs3.npy',test_caVFs)

np.save('train_moreleads_noseg_labels3.npy',train_elabels)
np.save('test_moreleads_noseg_labels3.npy',test_elabels)